In [35]:
from pathlib import Path
from textblob import TextBlob
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

BASE_PATH = Path('..')
events_path = BASE_PATH / 'events'
dictionary_path = BASE_PATH / 'dictionary'
data_path = BASE_PATH / 'data'
subset_reports_path = data_path / 'subset'
subset_reports_path_txt = data_path / 'subset_txt'
df_path = data_path / 'dataframes'
patterns_path = dictionary_path / 'patterns'
triggers_path = dictionary_path / 'trigger phrases'

GROUP = 0
group_events_path = events_path / f'group_{GROUP}_events.csv'
labelled_path = events_path / f'group_{GROUP}_labelled.csv'
processed_path = events_path / f'group_{GROUP}_processed.csv'

In [33]:
#Read in labelled event data file from 6 groups

groups = [0, 1, 2, 3, 4, 6] # lol

filenames = {group: events_path / f'group_{group}_labelled.csv' for group in groups}

# instantiate empty list to store dfs on read
dfall = []
for group in groups:
    df = pd.read_csv(filenames[group])
    
    # data processing and cleaning on near miss event column
    df = df.loc[df['Near Miss Event'].notna(), ]
    
    # pd.Series(['False']) returns True as string are converted to bool on whether they are empty or not!
    df['Near Miss Event'] = df['Near Miss Event'].apply(lambda x : (x == 'True') | (x == True)).astype(bool)
    
    # need to read in dataframe to work out length of group column
    df.insert(2, 'group', np.repeat(group, len(df)))
    dfall.append(df)
    
# concat list of dfs as a single data frame
dfall = pd.concat(dfall)

In [82]:
sid = SentimentIntensityAnalyzer()
score = sid.polarity_scores(sentence)


def nltk_scores(text):
    score = sid.polarity_scores(text)
    negative = score['neg']
    neutral = score['neu']
    positive = score['pos']
    compound = score['compound']
    return [negative, neutral, positive, compound]


{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


0.0

In [83]:
dfall['polarity'] = dfall.event_text.apply(lambda text : TextBlob(text).sentiment.polarity)
dfall['subjectivity'] = dfall.event_text.apply(lambda text : TextBlob(text).sentiment.subjectivity)
dfall['negative'] = dfall.event_text.apply(lambda text : nltk_scores(text)[0])
dfall['neutral'] = dfall.event_text.apply(lambda text : nltk_scores(text)[1])
dfall['positive'] = dfall.event_text.apply(lambda text : nltk_scores(text)[2])
dfall['compound'] = dfall.event_text.apply(lambda text : nltk_scores(text)[3])

In [85]:
dfall.to_csv(events_path / f'group_all_labelled_with_sentiment.csv', index=False)